# Deploy Agent To Vertex AI Agent Engine

[Vertex AI Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)
- [Deploy](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/deploy)
- [Use](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/adk)
- [Mangage](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/manage/overview)

[ADK Deployment](https://google.github.io/adk-docs/deploy/)
- [Vertex AI Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/)

Notes:
- This notebook expects the current working directory to contain the agents folder. In other words, `agent.py` is at `./document_agent/agent.py`.

In [1]:
!pwd

/home/statmike/repos/vertex-ai-mlops/Applied ML/Solution Prototypes/document-processing/7-agents/document_agent


In [29]:
!ls

apps  deploy-vertex-ai-agent-engine.ipynb  document_agent


In [2]:
!which python

/home/statmike/repos/vertex-ai-mlops/Applied ML/Solution Prototypes/document-processing/7-agents/.venv/bin/python


In [31]:
import os

from vertexai.preview import reasoning_engines
from vertexai import agent_engines
import vertexai
from dotenv import load_dotenv

In [28]:
gcs_path = 'applied-ml-solution-prototypes/document-processing/agent_engine/staging'

In [34]:
load_dotenv(dotenv_path = os.path.join('document_agent', '.env'))

True

In [39]:
vertexai.init(
    project = os.getenv('GOOGLE_CLOUD_PROJECT'),
    location = os.getenv('GOOGLE_CLOUD_LOCATION'),
    staging_bucket = f"gs://{os.getenv('GOOGLE_CLOUD_STORAGE_BUCKET')}"
)

---
## Local Testing: Prepare Agent for Vertex AI Agent Engine

In [5]:
from document_agent.agent import root_agent

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [6]:
session = app.create_session(user_id = 'u_123')
session

Session(id='586ff8e7-7278-475d-b940-3e6c712b8fa7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749296747.954514)

In [9]:
app.list_sessions(user_id = 'u_123')

ListSessionsResponse(sessions=[Session(id='586ff8e7-7278-475d-b940-3e6c712b8fa7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749296747.954514)])

In [10]:
session = app.get_session(user_id="u_123", session_id=session.id)
session

Session(id='586ff8e7-7278-475d-b940-3e6c712b8fa7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749296747.954514)

In [ ]:
message = []

In [12]:
for event in app.stream_query(
    user_id = 'u_123',
    session_id = session.id,
    message = 'What do you do?'
):
    print(event)

{'content': {'parts': [{'text': 'I am the primary agent that processes document related requests. I can help you extract information from documents, classify them, and compare them to vendor templates. To get started, please upload a PDF or PNG file, or provide a GCS URI (bucket and path) for the document you want to process.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 62, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 62}], 'prompt_token_count': 3474, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 3474}], 'total_token_count': 3536, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-b9c2c241-6b41-4048-b229-674fa8ed1a72', 'author': 'document_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'YVcq6fim', 'timestamp': 1749297058.693999}


In [14]:
for event in app.stream_query(
    user_id = 'u_123',
    session_id = session.id,
    message = 'use this file: gs://statmike-mlops-349915/applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf'
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-ecdaedea-aa07-4449-a69b-04b27a55e553', 'args': {'gcs_file_path': 'applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf', 'gcs_bucket': 'statmike-mlops-349915'}, 'name': 'get_gcs_file'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 56, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 56}], 'prompt_token_count': 3586, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 3586}], 'total_token_count': 3642, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-cf579d55-dc89-4d43-b132-0dd9902a0acf', 'author': 'document_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'uUHZbR1M', 'timestamp': 1749297194.573745}
{'content': {'parts': [{'function_response': {'id': 'adk-ecdaedea-aa07-4449-a69b-04b27a55e553', 'n

In [15]:
event

{'content': {'parts': [{'text': 'Okay, I have loaded the file from GCS. The artifact key for this document is `gcsfile_statmike-mlops-349915_applied-ml-solution-prototypes_document-processing_vendor_2_fake_invoices_vendor_2_invoice_10.pdf`. What would you like to do next? I can extract data from it, classify it, or compare it to a vendor template.\n'}],
  'role': 'model'},
 'usage_metadata': {'candidates_token_count': 94,
  'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
    'token_count': 94}],
  'prompt_token_count': 3754,
  'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
    'token_count': 3754}],
  'total_token_count': 3848,
  'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>},
 'invocation_id': 'e-cf579d55-dc89-4d43-b132-0dd9902a0acf',
 'author': 'document_agent',
 'actions': {'state_delta': {},
  'artifact_delta': {},
  'requested_auth_configs': {}},
 'id': 'LRobMLTu',
 'timestamp': 1749297197.823944}

In [23]:
app.get_session(user_id = 'u_123', session_id = session.id)

Session(id='586ff8e7-7278-475d-b940-3e6c712b8fa7', app_name='default-app-name', user_id='u_123', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='What do you do?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-b9c2c241-6b41-4048-b229-674fa8ed1a72', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='O1XFSeCz', timestamp=1749297058.690996), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executa

In [22]:
for event in app.get_session(user_id = 'u_123', session_id = session.id).events:
    print(event)

content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='What do you do?')], role='user') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None invocation_id='e-b9c2c241-6b41-4048-b229-674fa8ed1a72' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}) long_running_tool_ids=None branch=None id='O1XFSeCz' timestamp=1749297058.690996
content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='I am the primary agent that processes document related requests. I can help you

more
- delete session
- create session with specified id
- 

---
## Deploy To Vertex AI Agent Engine

In [45]:
remote_app = agent_engines.create(
    agent_engine = root_agent,
    requirements = "../requirements.txt",
    extra_packages = ["./document_agent"],
    gcs_dir_name = gcs_path,
    display_name = root_agent.name,
    description = root_agent.description,
    #env_vars = 
)

Deploying google.adk.agents.Agent as an application.
Reading requirements from requirements='../requirements.txt'
Read the following lines: ['absl-py==2.3.0', 'aiofiles==24.1.0', 'altair==5.5.0', 'annotated-types==0.7.0', 'anyio==4.9.0', 'asttokens==3.0.0', 'attrs==25.3.0', 'audioop-lts==0.2.1', 'Authlib==1.5.2', 'blinker==1.9.0', 'cachetools==5.5.2', 'certifi==2025.4.26', 'cffi==1.17.1', 'charset-normalizer==3.4.1', 'click==8.1.8', 'cloudpickle==3.1.1', 'comm==0.2.2', 'cryptography==44.0.2', 'db-dtypes==1.4.3', 'debugpy==1.8.14', 'decorator==5.2.1', 'deepdiff==6.7.1', 'Deprecated==1.2.18', 'docstring_parser==0.16', 'executing==2.2.0', 'fastapi==0.115.12', 'ffmpy==0.5.0', 'filelock==3.18.0', 'Flask==3.1.1', 'fsspec==2025.5.1', 'gitdb==4.0.12', 'GitPython==3.1.44', 'google-adk==1.2.1', 'google-api-core==2.24.2', 'google-api-python-client==2.169.0', 'google-auth==2.39.0', 'google-auth-httplib2==0.2.0', 'google-cloud-aiplatform==1.96.0', 'google-cloud-appengine-logging==1.6.1', 'google-cl

In [46]:
remote_app.resource_name

'projects/1026793852137/locations/us-central1/reasoningEngines/6576847548797943808'

In [47]:
remote_session = remote_app.create_session(user_id = 'u_123')
remote_session

{'id': '5496252468822016000',
 'events': [],
 'lastUpdateTime': 1749306781.997659,
 'state': {},
 'appName': '6576847548797943808',
 'userId': 'u_123'}

In [49]:
remote_app.list_sessions(user_id = 'u_123')

{'sessions': [{'id': '5496252468822016000',
   'lastUpdateTime': 1749306781.997659,
   'state': {},
   'events': [],
   'appName': '6576847548797943808',
   'userId': 'u_123'}]}

In [50]:
remote_session = remote_app.get_session(user_id = "u_123", session_id = remote_session['id'])
remote_session

{'id': '5496252468822016000',
 'state': {},
 'lastUpdateTime': 1749306781.997659,
 'events': [],
 'appName': '6576847548797943808',
 'userId': 'u_123'}

In [51]:
for event in remote_app.stream_query(
    user_id = 'u_123',
    session_id = remote_session['id'],
    message = 'What do you do?'
):
    print(event)

{'content': {'parts': [{'text': 'I am the primary agent that processes document-related requests. I can load documents, extract information from them, classify them, and compare them to vendor templates. To start, please upload a PDF or PNG file, or provide a GCS URI (bucket and path) for the document you want to process.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 63, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 63}], 'prompt_token_count': 3474, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 3474}], 'total_token_count': 3537, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-03680743-4c36-41b6-8131-b3446cd9f724', 'author': 'document_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'Im4zfnmS', 'timestamp': 1749306984.444021}


In [52]:
for event in remote_app.stream_query(
    user_id = 'u_123',
    session_id = remote_session['id'],
    message = 'use this file: gs://statmike-mlops-349915/applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf'
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-8bca8c77-0a42-4597-b4d0-3066e6a23aae', 'args': {'gcs_file_path': 'applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf', 'gcs_bucket': 'statmike-mlops-349915'}, 'name': 'get_gcs_file'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 56, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 56}], 'prompt_token_count': 3587, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 3587}], 'total_token_count': 3643, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-82d29f0e-e7a8-49cc-a555-671c9401e1c7', 'author': 'document_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'v8RDV5z5', 'timestamp': 1749307022.765931}
{'content': {'parts': [{'function_response': {'id': 'adk-8bca8c77-0a42-4597-b4d0-3066e6a23aae', 'name': 'get_gcs_file', 'response': {'result': 'The file vendor_2_invoice_

In [53]:
event

{'content': {'parts': [{'text': 'OK. I have loaded the file from GCS. The artifact key is `gcsfile_statmike-mlops-349915_applied-ml-solution-prototypes_document-processing_vendor_2_fake_invoices_vendor_2_invoice_10.pdf`.\n'}],
  'role': 'model'},
 'usage_metadata': {'candidates_token_count': 65,
  'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 65}],
  'prompt_token_count': 3755,
  'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 3755}],
  'total_token_count': 3820,
  'traffic_type': 'ON_DEMAND'},
 'invocation_id': 'e-82d29f0e-e7a8-49cc-a555-671c9401e1c7',
 'author': 'document_agent',
 'actions': {'state_delta': {},
  'artifact_delta': {},
  'requested_auth_configs': {}},
 'id': 'fejBGJ9q',
 'timestamp': 1749307024.585193}

In [54]:
remote_app.get_session(user_id = 'u_123', session_id = remote_session['id'])

{'id': '5496252468822016000',
 'events': [{'errorMessage': None,
   'partial': None,
   'invocationId': 'e-03680743-4c36-41b6-8131-b3446cd9f724',
   'branch': None,
   'id': '7008703280595337216',
   'turnComplete': None,
   'usageMetadata': None,
   'content': {'role': 'user',
    'parts': [{'fileData': None,
      'thought': None,
      'codeExecutionResult': None,
      'text': 'What do you do?',
      'inlineData': None,
      'videoMetadata': None,
      'thoughtSignature': None,
      'functionResponse': None,
      'executableCode': None,
      'functionCall': None}]},
   'groundingMetadata': None,
   'customMetadata': None,
   'longRunningToolIds': None,
   'errorCode': None,
   'timestamp': 1749306984.110913,
   'author': 'user',
   'interrupted': None,
   'actions': {'artifactDelta': {},
    'escalate': None,
    'skipSummarization': None,
    'transferToAgent': None,
    'requestedAuthConfigs': {},
    'stateDelta': {}}},
  {'errorMessage': None,
   'partial': None,
   'invo

In [56]:
for event in remote_app.get_session(user_id = 'u_123', session_id = remote_session['id'])['events']:
    print(event)

{'interrupted': None, 'partial': None, 'invocationId': 'e-03680743-4c36-41b6-8131-b3446cd9f724', 'branch': None, 'id': '7008703280595337216', 'turnComplete': None, 'usageMetadata': None, 'content': {'role': 'user', 'parts': [{'fileData': None, 'thought': None, 'codeExecutionResult': None, 'inlineData': None, 'text': 'What do you do?', 'videoMetadata': None, 'thoughtSignature': None, 'functionResponse': None, 'executableCode': None, 'functionCall': None}]}, 'groundingMetadata': None, 'customMetadata': None, 'longRunningToolIds': None, 'timestamp': 1749306984.110913, 'errorCode': None, 'author': 'user', 'errorMessage': None, 'actions': {'artifactDelta': {}, 'escalate': None, 'transferToAgent': None, 'skipSummarization': None, 'requestedAuthConfigs': {}, 'stateDelta': {}}}
{'errorMessage': None, 'partial': None, 'invocationId': 'e-03680743-4c36-41b6-8131-b3446cd9f724', 'branch': None, 'id': '954176531549126656', 'turnComplete': None, 'usageMetadata': None, 'content': {'role': 'model', 'pa

more
- delete session
- create session with specified id
- delete app